# Syntax for computing with components

In [38]:
from compute import Var
from datastructures.execution import Component

## Component handles unit conversions

In [2]:
Pcomms = Var('P_{comms}', unit='W')
D = Var('D', unit='m')
h = Var('h', unit='km')
R = Var('R', unit='kW')
sizing_relationship = Pcomms*D**2/h**2*1e18/(8e6)

In [3]:
sizing_relationship

125000000000.0*D**2*P_{comms}/h**2

In [4]:
component = Component.fromsympy(sizing_relationship, tovar=R)

## Component attributes

In [5]:
component.inputs, component.indims

(('h', 'P_{comms}', 'D'), (1, 1, 1))

In [6]:
component.outputs, component.outdims

(('R',), (1,))

## Evaluate components

In [7]:
component.evaldict({'h':400, 'P_{comms}':10, 'D':10})

[DeviceArray(0.78125, dtype=float64)]

## Evaluate gradients through automatic differentiation

In [8]:
component.graddict({'h':400, 'P_{comms}':10, 'D':10})

{('R', 'h'): DeviceArray(-0.00390625, dtype=float64),
 ('R', 'P_{comms}'): DeviceArray(0.078125, dtype=float64),
 ('R', 'D'): DeviceArray(0.15625, dtype=float64)}

## Component manual creation
Demonstration with matrix/vector quantities

In [27]:
import numpy as np
A = np.random.rand(10,10)
b0 = np.random.rand(10)
x0 = np.random.rand(10)
fx = lambda x,b: (b@(A@x-b),) # The @ is numpys matrix multiplication
component = Component(fx, inputs=('x','b'), outputs=('y',), indims=(10,10), outdims=(1,))

In [28]:
component.evaldict({'x':x0, 'b':b0})

(4.391333423109129,)

In [29]:
component.graddict({'x':x0, 'b':b0})

{('y',
  'x'): DeviceArray([2.18488828, 1.43761381, 2.01779949, 1.61428775, 2.76793662,
              1.87935127, 2.02227283, 0.88950607, 1.8922517 , 2.036412  ],            dtype=float64),
 ('y',
  'b'): DeviceArray([1.0146121 , 1.13650587, 0.33435548, 1.81286402, 1.94674412,
              1.05914187, 1.0388783 , 0.71057041, 0.0901676 , 1.00682255],            dtype=float64)}

### Check against analytical derivatives

In [39]:
b0@A # dy/dx

array([2.18488828, 1.43761381, 2.01779949, 1.61428775, 2.76793662,
       1.87935127, 2.02227283, 0.88950607, 1.8922517 , 2.036412  ])

In [40]:
A@x0-2*b0 # dy/db

array([1.0146121 , 1.13650587, 0.33435548, 1.81286402, 1.94674412,
       1.05914187, 1.0388783 , 0.71057041, 0.0901676 , 1.00682255])